In [ ]:
# 分割出 ROI 並計算植生指標 (NDVI, NDRE, NDI, RGI, ExG)

# 讀取檔案用
import os
import glob
from pathlib import Path

# 分析影像用
from osgeo import gdal, ogr
from skimage import io, filters # 用於 Otsu 大津演算法
import shapefile
import numpy as np
import matplotlib.pyplot as plt

# 輸出 csv 用
import csv

# 顯示處理進度與計時用
from tqdm import tqdm, trange
import threading
import time 
from datetime import datetime

# 引用同一資料夾檔案內 function
from utilis import *

# 建立植生指標名稱 list，用來建立圖層儲存迴圈
VIs = ['NDVI', 'NDRE','NDI','RGI','ExG']

# 輸出 csv 欄位名稱
fieldnames = ['Id', 'site', 'date', 'Period', 'B', 'G', 'R', 'RE', 'NIR', 'GC'] + [f'{VI}' for VI in VIs] + [f'{VI}_sum' for VI in VIs] + [f'{VI}_max' for VI in VIs] + [f'{VI}_min' for VI in VIs] + [f'{VI}_range' for VI in VIs] + [f'{VI}_std' for VI in VIs]

# 給定開始時間
st_time_total = time.time()

# 判斷是否存在 data 與 images 資料夾
dir_maker(f'03_Data/')
dir_maker(f'04_Result/Image')

now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

with open(f'03_Data/{now}_ROI_Result.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    #寫入欄位名稱，key值
    writer.writeheader()

# 開啟 csv 檔案 (新增資料格式)
with open(f'03_Data/{now}_ROI_Result.csv', 'a', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)

    # 使用 glob.glob 獲取檔案列表
    image_paths = glob.glob('01_Tif/6band/UC/*band*.tif')

    # 使用 tqdm 包裹檔案列表，顯示進度條
    for i, image_path in enumerate(image_paths, 1):  # enumerate 從 1 開始計數
        print(f'第{i}個目標路徑：{image_path}')
        print('------------------------------')
        print(f"開始讀取第{i}個tif檔資訊")

        # 讀取 TIF 檔案時間
        st_time_read = time.time()

        # 建立要寫入 csv 的資料容器
        data = {}

        # Tif 檔名稱、讀取場域與日期
        tif_name = Path(image_path).stem
        site = Path(image_path).parts[-2]
        date = tif_name[:10].replace('-', '')
        Period = Which_Period(date)
        print(f'檔案名: {tif_name}，場域: {site}，日期: {date}，期作: {Period}')

        # 寫入初始資料
        data['site'] = site
        data['date'] = date
        data['Period'] = Period

        # 讀取 tif 原圖為 dataset
        dataset = gdal.Open(image_path, gdal.GA_ReadOnly)

        # 讀取 tif 原圖的影像資料，並且正規化
        img = dataset.ReadAsArray() / 65535

        # 將數值為 1(即原本為 65535 之值) 設定為 0
        img[img == 1.0] = 0

        # 讀取影像資料中的各波段，以進行植生指標運算
        B = img[0, :, :]
        G = img[1, :, :]
        R = img[2, :, :]
        RE = img[3, :, :]
        NIR = img[4, :, :]

        # 計算 NDVI 跟 NDRE
        NDVI = (NIR - R) / (NIR + R + 1e-10)
        NDRE = (NIR - RE) / (NIR + RE + 1e-10)
        NDI = (G - R) / (G + R + (10)**(-10))
        RGI =  R / (G + (10)**(-10))
        ExG = (2 * G - B - R) / (R + B + G + (10)**(-10))
        GBGAP = G - B

        # 分別使用大津演算法二值化
        thresh1 = filters.threshold_otsu(NIR)
        thresh2 = filters.threshold_otsu(NDVI)
        thresh3 = filters.threshold_otsu(NDRE)

        # 建立 mask
        mask1 = (NIR >= thresh1) * 1.0
        mask2 = (NDVI >= thresh2) * 1.0
        mask3 = (NDRE >= thresh3) * 1.0
        mask4 = (GBGAP >= 0) * 1.0
        mask5 = (R != 0) * (G !=0) * (B != 0) * 1.0
        
        mask = mask1 * mask2 * mask3 * mask4 * mask5

        dir_maker(f'04_Result/Image/{site}/ROI/{date}/')
        
        new_dataset = gdal.Translate(f'./04_Result/Image/{site}/ROI/{date}/mask.tif', dataset, bandList=[1], outputType=gdal.GDT_Float32) 
        new_dataset.WriteArray(mask)
        new_dataset = None

        print(f"第{i}個 tif 檔數值讀取完畢")
        print('------------------------------')

        # 讀取 TIF 檔案時間結束
        print(f'讀取 tif 檔案時間: {round(time.time() - st_time_read, 2)} 秒\n')

        # 把 ROI 的 shapefile 讀出來
        shapefile_path = glob.glob(f'./02_Shapefile/{site}/ROI/MERGE.shp')[0]   # 此處需要依實際情況修改路徑

        # 讀取整個 shapefile
        input_shapefile = ogr.Open(shapefile_path)
        input_layer = input_shapefile.GetLayer(0)

        # 針對每個日期每個區域，建立用 shapefile 切下來的 6band 檔案
        dir_maker(f'04_Result/Shapefile/{site}/ROI/{Period}')
        dir_maker(f'04_Result/Image/{site}/ROI/{date}/tif')

        # Shapefile 切割時間
        st_time_shapefile = time.time()

        shapefile_count = 0
        print('------------------------------')                
        print(f'{site}_{date} 開始切割 Shapefile 檔案')

        for shpfile in range(input_layer.GetFeatureCount()):
            try:
                feature = input_layer.GetFeature(shpfile)
                Id = feature.GetField('Id')
                Field = feature.GetField('Field')

                # 建立儲存路徑
                shp_output_path = f'04_Result/Shapefile/{site}/ROI/{Period}'
                
                # 建立檔案名稱，Field 名稱_Id 號碼
                save_shp_path = os.path.join(shp_output_path, f'{Field}_{Id}.shp')
                
                # 避免產生同樣名稱的 Shapefile
                if os.path.isfile(save_shp_path) or Field == 'temp':
                    tqdm.write(f"已有相同名稱的 Shapefile: {Field}_{Id}.shp")
                    continue
                
                driver = ogr.GetDriverByName('ESRI Shapefile')  # driver
                datasource = driver.CreateDataSource(save_shp_path)
                srs = input_layer.GetSpatialRef()  # 取得座標系統
                layer = datasource.CreateLayer('layername', srs, geom_type=ogr.wkbPolygon)
                layer.CreateFeature(feature)
                geometry = feature.GetGeometryRef()
                
                # 關閉資料源和圖層
                datasource.Destroy()
                
                # 計算產生的 Shapefile 數目
                shapefile_count += 1
                
            except Exception as e:
                tqdm.write(f"Error processing feature {shpfile}: {e}")
                pass

        # Shapefile 切割時間結束
        print(f'Shapefile切割時間: {round(time.time() - st_time_shapefile, 2)} 秒')
        print(f'共產生了 {shapefile_count} 塊 Shapefile\n')
        print('------------------------------')

        # 重新計時
        print(f'{site}_{date} 開始切割小塊的 tif 檔案')
        st_time_cut_tif = time.time()

        # 讀取該場域該日期所有的 shapefile 檔案
        shapefile_path_list = glob.glob(f'04_Result/Shapefile/{site}/ROI/{Period}/*.shp')

        # 建立小塊 tif 檔案切割進度條
        for i in trange(len(shapefile_path_list)):
            # 指定 shapefile 的路徑
            shp_path = shapefile_path_list[i]

            if Path(shp_path).stem == 'temp':
                continue

            # 小塊 tif 檔案的輸出路徑
            tif_file_output_path = f'04_Result/Image/{site}/ROI/{date}/tif/' + Path(shp_path).stem + '.tif'
            
            # 獲得輸入 shapefile 的座標資料
            r = shapefile.Reader(shp_path)
            minX, minY, maxX, maxY = r.bbox

            ds = gdal.Warp(
                tif_file_output_path,
                image_path,
                format='GTiff',
                cutlineDSName=shp_path,
                outputBounds=[minX, minY, maxX, maxY],
                dstNodata=0,
            )

        print(f'{site}_{date} 小塊 tif 切割結束，花費時間 {round(time.time() - st_time_cut_tif, 2)} 秒\n')
        print('------------------------------')

        # 建立該場域該日期的所有樣區的 tif 檔案路徑 list
        tif_file_path_list = glob.glob(f'04_Result/Image/{site}/ROI/{date}/tif/*.tif')

        # 建立分析植生指標的進度條
        for i in range(len(tif_file_path_list)):
            # 指定要分析的 tif 檔案
            tif_file_path = tif_file_path_list[i]

            # 該 tif 檔案名稱
            tif_name = Path(tif_file_path).stem
            data['Id'] = tif_name

            # 用檔案名稱把資訊讀出來
            Id = tif_name.split("_")[0]

            # 讀取原圖 tif 檔的資訊
            dataset = gdal.Open(tif_file_path, gdal.GA_ReadOnly)

            # 讀取 tif 檔案原圖圖片資訊
            image = dataset.ReadAsArray() / 65535
            image[image == 1.0] = 0

            # 讀取所有 band
            B = image[0, :, :]
            G = image[1, :, :]
            R = image[2, :, :]
            RE = image[3, :, :]
            NIR = image[4, :, :]

            # 計算 NDVI 跟 NDRE
            NDVI = (NIR - R) / (NIR + R + 1e-10)
            NDRE = (NIR - RE) / (NIR + RE + 1e-10)
            NDI = (G - R) / (G + R + (10)**(-10))
            RGI =  R / (G + (10)**(-10))
            ExG = (2 * G - B - R) / (R + B + G + (10)**(-10))
            GBGAP = G - B

            # 建立 mask
            mask1 = (NIR >= thresh1) * 1.0
            mask2 = (NDVI >= thresh2) * 1.0
            mask3 = (NDRE >= thresh3) * 1.0
            mask4 = (GBGAP >= 0) * 1.0
            mask5 = (R != 0) * (G !=0) * (B != 0) * 1.0
            
            mask = mask1 * mask2 * mask3 * mask4 * mask5

            dir_maker(f'04_Result/Image/{site}/ROI/{date}/')
            
            new_dataset = gdal.Translate(f'04_Result/Image/{site}/ROI/{date}/mask.tif', dataset, bandList=[1], outputType=gdal.GDT_Float32) 
            new_dataset.WriteArray(mask)
            new_dataset = None

            # 計算冠層覆蓋率，運算邏輯：計算原圖(image)的隨便一個波段所有非零像素植體的數量，再除以原圖該波段的所有非零像素植體數量
            GC_area = np.count_nonzero(image[0, :, :] * mask)
            total_area = np.count_nonzero(image[0, :, :])
            GC_rate = (GC_area / total_area)

            B = B * mask
            G = G * mask
            R = R * mask
            RE = RE * mask
            NIR = NIR * mask

            data['B'] = np.nanmean(B)
            data['G'] = np.nanmean(G)
            data['R'] = np.nanmean(R)
            data['RE'] = np.nanmean(RE)
            data['NIR'] = np.nanmean(NIR)
            data['GC'] = GC_rate

            # 計算 NDVI 和 NDRE 數列的非 nan 平均值，如果還是 nan 值，則平均值與其他統計量一樣設定為 0
            for VI in VIs:
                st_time_vi = time.time()  # 開始計時
                VI_img = eval(VI)
                VI_img_nozero = VI_img[VI_img != 0.0]
                if VI == "RGI":
                    VI_img_nozero = VI_img_nozero[VI_img_nozero <= 1.0]

                data[f'{VI}'] = np.nanmean(VI_img_nozero)
                if str(data[f'{VI}']) == 'nan':
                    data[f'{VI}'] = 0
                    data[f'{VI}_sum'] = 0
                    data[f'{VI}_max'] = 0
                    data[f'{VI}_min'] = 0
                    data[f'{VI}_range'] = 0
                    data[f'{VI}_std'] = 0

                else:
                    data[f'{VI}_sum'] = np.nansum(VI_img_nozero)
                    data[f'{VI}_max'] = np.nanmax(VI_img_nozero)
                    data[f'{VI}_min'] = np.nanmin(VI_img_nozero)
                    data[f'{VI}_range'] = np.nanmax(VI_img_nozero) - np.nanmin(VI_img_nozero)
                    data[f'{VI}_std'] = np.nanstd(VI_img_nozero)

                # 確認輸出路徑資料夾是否存在
                dir_maker(f'04_Result/Image/{site}/ROI/{date}/VI/')

                try:
                    # 設定要儲存的 tif 檔案路徑，以原圖的 dataset 設定，並放入一個 band，輸出格式為 float32
                    new_dataset = gdal.Translate(f'04_Result/Image/{site}/ROI/{date}/VI/{VI}_{tif_name}.tif', dataset, bandList=[1], outputType=gdal.GDT_Float32) 
                    new_dataset.WriteArray(VI_img)
                    new_dataset = None
                except:
                    pass

                # 計算和輸出 VI 時間
                print(f'{site}_{date}_{tif_name}_{VI} 處理時間: {round(time.time() - st_time_vi, 2)} 秒')
                print('------------------------------')

            # 將一小塊 tif 檔案的分析結果寫入 csv 檔案中
            writer.writerow(data)
        
# 分析完畢，計時
print(f'{site} 分析結束，共分析 {len(image_paths)} 個檔案，花費時間 {round(time.time() - st_time_total, 2)} 秒')   

In [7]:
# 合併 VI 資料內的各個植生指標的 tif 檔
import os
import time
from osgeo import gdal
from utilis import *

# 定義植生指標
VIs = ['NDVI', 'NDRE','NDI','RGI','ExG']

# 定義主要輸入文件夾
main_input_folder = '04_Result/Image/UC/ROI'
output_base_folder = '04_Result/Image/UC/ROI'

# 記錄總開始時間
total_start_time = time.time()

# 遍歷 ROI 資料夾中的所有日期資料夾
for i, date in enumerate(os.listdir(main_input_folder), 1):
    input_folder = os.path.join(main_input_folder, date, 'VI')
    output_folder = os.path.join(output_base_folder, date, 'VI', 'Merged')
    print('------------------------------')
    print(f"開始處理第 {i} 個資料夾，日期為:{date}")
    dir_maker(output_folder)

    # 合併每個指標的 TIF 文件並計算時間
    for VI in VIs:
        start_time = time.time()
        merge_tif_files(VI, input_folder, output_folder)
        end_time = time.time()
        
        print(f'{date}_{VI} 合併花費時間: {end_time - start_time:.2f} 秒')

    print(f"第 {i} 個資料夾處理結束")
    print('------------------------------')
    print()

# 計算並顯示總時間
total_end_time = time.time()
print(f'合併完成，共花費時間: {total_end_time - total_start_time:.2f} 秒')

------------------------------
開始處理第 1 個資料夾，日期為:20240415
新增此路徑: ./04_Result/Image/UC/ROI/20240415/VI/Merged
20240415_NDVI 合併花費時間: 3.05 秒
20240415_NDRE 合併花費時間: 3.10 秒
20240415_NDI 合併花費時間: 3.37 秒
20240415_RGI 合併花費時間: 12.58 秒
20240415_ExG 合併花費時間: 3.98 秒
第 1 個資料夾處理結束
------------------------------

------------------------------
開始處理第 2 個資料夾，日期為:20240513
新增此路徑: ./04_Result/Image/UC/ROI/20240513/VI/Merged
20240513_NDVI 合併花費時間: 3.44 秒
20240513_NDRE 合併花費時間: 13.33 秒
20240513_NDI 合併花費時間: 3.51 秒
20240513_RGI 合併花費時間: 3.48 秒
20240513_ExG 合併花費時間: 13.35 秒
第 2 個資料夾處理結束
------------------------------

------------------------------
開始處理第 3 個資料夾，日期為:20240516
新增此路徑: ./04_Result/Image/UC/ROI/20240516/VI/Merged
20240516_NDVI 合併花費時間: 10.57 秒
20240516_NDRE 合併花費時間: 11.88 秒
20240516_NDI 合併花費時間: 11.18 秒
20240516_RGI 合併花費時間: 11.60 秒
20240516_ExG 合併花費時間: 10.62 秒
第 3 個資料夾處理結束
------------------------------

------------------------------
開始處理第 4 個資料夾，日期為:20240520
新增此路徑: ./04_Result/Image/UC/ROI/20240520/VI/Merged